# A Course on Data Wrangling and Visualization

Nelson Uhan

Math Department Teaching Seminar<br>
3 December 2020

- This semester, Jay Foraker and I developed a new course on data wrangling and visualization<br><br>

- What is this course about?<br><br>

- Structured ways to think about data wrangling and visualization

## Motivation for the course

- The OR curriculum exposes students to a wide variety of modeling and algorithmic techniques

- These concepts have been typically taught with small, tidy data sets

- Unfortunately, as a result, students have not been well-equipped to tackle the large, messy data sets typically involved with OR capstone projects

- This course aims to fill this gap

## Student goals for the course

1. Learn to create useful visualizations of data through a **grammar of graphics**<br><br>

2. Learn to wrangle (i.e. clean and manipulate) large, messy data sets into forms suitable for modeling and analysis through a **grammar of data manipulation**<br><br>

3. Increase general fluency with Python

## Grammar of graphics

- A **grammar of graphics** allows us to concisely describe the components of a (statistical) visualization<br><br>

- With such a grammar, we can move beyond named graphics (e.g. "scatterplot", "bar graph") and specify basic and complex visualizations in a structured way

- Popularized by [Hadley Wickham](http://hadley.nz/) (Chief Scientist, R Studio) and his [ggplot2](https://ggplot2.tidyverse.org/) package for R<br><br>

- In this course, we used [Altair](https://altair-viz.github.io/), a grammar-based visualization package for Python

## Components of a visualization

*Note.* This is Altair's terminology.

- **Dataset** with variables and observations

- **Encoding channels** map variables to visual attributes (e.g. x-position, y-position, color, shape)
    - **Scales** for each encoding channel to adjust these mappings (e.g. linear vs. log scale positions)

- **Graphical marks** specify how these visual attributes should be visually represented (e.g. points, lines, bars)

- **Transformations** modify the data before visualization

- **Layering**, **concatenation**, and **faceting** specify how to combine or generate multiple related charts

## An example

- Let's use some health and population data for a number of countries between 1955 and 2005
    - Data from by the [Gapminder Foundation](https://www.gapminder.org/)

- First, let's take a look at the first 5 rows of the data:


<table border="1" class="dataframe">
  <thead>
    <tr style="text-align: right;">
      <th></th>
      <th>year</th>
      <th>country</th>
      <th>cluster</th>
      <th>pop</th>
      <th>life_expect</th>
      <th>fertility</th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <th>0</th>
      <td>1955</td>
      <td>Afghanistan</td>
      <td>South Asia</td>
      <td>8891209</td>
      <td>30.332</td>
      <td>7.7</td>
    </tr>
    <tr>
      <th>1</th>
      <td>1960</td>
      <td>Afghanistan</td>
      <td>South Asia</td>
      <td>9829450</td>
      <td>31.997</td>
      <td>7.7</td>
    </tr>
    <tr>
      <th>2</th>
      <td>1965</td>
      <td>Afghanistan</td>
      <td>South Asia</td>
      <td>10997885</td>
      <td>34.020</td>
      <td>7.7</td>
    </tr>
    <tr>
      <th>3</th>
      <td>1970</td>
      <td>Afghanistan</td>
      <td>South Asia</td>
      <td>12430623</td>
      <td>36.088</td>
      <td>7.7</td>
    </tr>
    <tr>
      <th>4</th>
      <td>1975</td>
      <td>Afghanistan</td>
      <td>South Asia</td>
      <td>14132019</td>
      <td>38.438</td>
      <td>7.7</td>
    </tr>
  </tbody>
</table>

- Each row of this dataset contains the following data for each `country` and `year`:
    - region of the world (`cluster`)
    - total population (`pop`)
    - average life expectancy in years (`life_expect`)
    - number of children per woman (`fertility`)

- Let's use Altair to plot the average life expectancy (`life_expect`) vs. number of children per woman (`fertility`) in the year 2000

In [1]:
import altair as alt

data_url = 'data/gapminder.csv'

In [2]:
alt.Chart(data_url).transform_filter(
    'datum.year == 2000'
).mark_point(filled=True).encode(
    alt.X('fertility:Q'),
    alt.Y('life_expect:Q'),
    alt.Size('pop:Q'),
    alt.Color('cluster:N'),
    alt.Tooltip('country:N')
)

alt.Chart(...)

- What if we wanted to create a similar chart for every year in the dataset?<br><br>

- We can use **faceting**:

In [3]:
base = alt.Chart().mark_point(filled=True).encode(
    alt.X('fertility:Q'),
    alt.Y('life_expect:Q'),
    alt.Size('pop:Q'),
    alt.Color('cluster:N'),
    alt.Tooltip('country:N')
).properties(
    width=150,
    height=150
)

base.facet(
    data=data_url,
    facet=alt.Facet('year:N'),
    columns=6
)

alt.FacetChart(...)

## Grammar of data manipulation

- A **grammar of data manipulation** allows us to use a consistent set of operations to solve the most common data manipulation challenges

- [dplyr](https://dplyr.tidyverse.org/) (also by Hadley Wickham) is a R package that embodies this notion
    - One function per operation
    - Inspired by SQL

- In this course, we used [Pandas](https://pandas.pydata.org/), the popular Python data manipulation library<br><br>

- Unfortunately, Pandas is notorious for having multiple ways of doing the same operation<br><br>

- For this course, we restricted ourselves to an *opinionated subset* of Pandas, with one way to achieve each operation

##  Basic operations for data manipulation

- **Filter rows** based on their values

- **Select and drop columns** based on their names

- **Sort rows** based on their values

- **Create new columns** that are functions of existing columns

- Aggregate, transform, and filter **groups of data** through **split-apply-combine**

- **Pivot** data from long form to wide form and vice versa

- **Merge** datasets together based on key columns

<img src="img/aggregation.jpg" width="70%" />

| Operation | Opinionated Pandas | dplyr |
| :- | :- | :- |
| Filter rows  | `.query()` | `filter()` |
| Select and drop columns | `[[...]]` and `.drop()` | `select()` |
| Sorting rows | `.sort_values()` | `arrange()` |
| Create new columns | `.assign()` | `mutate()` |
| Aggregate groups of data | `.groupby()` with `.agg()` | `group_by()` with `summarize()` |

## An example

- Let's read the data into a Pandas DataFrame:

In [4]:
import pandas as pd

df = pd.read_csv('data/gapminder.csv')
df.head()

,year,country,cluster,pop,life_expect,fertility
0,1955,Afghanistan,South Asia,8891209,30.332,7.7
1,1960,Afghanistan,South Asia,9829450,31.997,7.7
2,1965,Afghanistan,South Asia,10997885,34.020,7.7
3,1970,Afghanistan,South Asia,12430623,36.088,7.7
4,1975,Afghanistan,South Asia,14132019,38.438,7.7


- Let's find the country with the highest life-expectancy-to-fertility ratio in each year between 1980 to 2000 in the dataset.

In [5]:
(
    df
    .assign(
        ratio=lambda x: x['life_expect'] / x['fertility']
    )
    .sort_values(['year', 'ratio'], ascending=[True, False])
    .groupby('year')
    .agg(
        highest_ratio_country=('country', 'first'),
        highest_ratio=('ratio', 'first')
    )
    .reset_index()    
    .query('year >= 1980 and year <= 2000')

)

,year,highest_ratio_country,highest_ratio
5,1980,Germany,50.547945
6,1985,Hong Kong,58.167939
7,1990,Spain,61.078740
8,1995,Hong Kong,74.074074
9,2000,Hong Kong,86.696809


## Course Projects

**Project 1.** Visually explore the relationships between colleges, majors, and salary after graduation, using data from [PayScale](https://payscale.com)

<img src="img/major_midcareer_pay.svg" width="75%">

**Project 2.** Practice layering and customizing Altair charts by reproducing this graphic showing the relationship between corruption and human development, originally published in *The Economist* in 2011.

<img src="img/economist_altair.svg" width="55%">

**Project 3.** Create several visualizations of COVID-19 in the United States, using a dataset created by the instructors, based on COVID-19 cases and deaths data from [usafacts.org](https://usafacts.org/visualizations/coronavirus-covid-19-spread-map/), combined with geographic data from the [United States Census Bureau](https://www.census.gov/geographies/reference-files/time-series/geo/gazetteer-files.html).

<img src="img/problem1.png" width="50%">

**Project 5.** Use Pandas to recreate the dataset used in Project 3.

**Project 4.** Use Pandas to explore [this Kaggle dataset](https://www.kaggle.com/yamaerenay/spotify-dataset-19212020-160k-tracks) containing the following information on over 160,000 tracks on Spotify, including measures such as *acousticness*, *danceability*, *energy*, *instrumentalness*, *liveness*, and *speechiness*.

<table border="1" class="dataframe">
  <thead>
    <tr style="text-align: right;">
      <th></th>
      <th>decade</th>
      <th>median_danceability</th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <th>10</th>
      <td>2020</td>
      <td>0.693</td>
    </tr>
    <tr>
      <th>0</th>
      <td>1920</td>
      <td>0.624</td>
    </tr>
    <tr>
      <th>9</th>
      <td>2010</td>
      <td>0.612</td>
    </tr>
    <tr>
      <th>7</th>
      <td>1990</td>
      <td>0.587</td>
    </tr>
    <tr>
      <th>8</th>
      <td>2000</td>
      <td>0.583</td>
    </tr>
    <tr>
      <th>6</th>
      <td>1980</td>
      <td>0.564</td>
    </tr>
    <tr>
      <th>1</th>
      <td>1930</td>
      <td>0.558</td>
    </tr>
    <tr>
      <th>5</th>
      <td>1970</td>
      <td>0.530</td>
    </tr>
    <tr>
      <th>4</th>
      <td>1960</td>
      <td>0.507</td>
    </tr>
    <tr>
      <th>3</th>
      <td>1950</td>
      <td>0.489</td>
    </tr>
    <tr>
      <th>2</th>
      <td>1940</td>
      <td>0.470</td>
    </tr>
  </tbody>
</table>

**Project 6.** Work with a (perturbed) dataset on ordnance onload and offload operations performed by Navy Munitions Command Atlantic (NMCLANT) Detachment (Det) Sewells Point in 2015. Clean the data, compute performance metrics, create interactive visualizations of the data.

<img src="img/histogram_nmc.png" width="40%" />

## Other topics we had hoped to cover

- Getting data through **web scraping**

- Getting data through **website APIs**

- Interoperability between R and Python
    - [rpy2](https://rpy2.github.io/doc.html) is a Python library that lets you call R directly from inside Python
    - [reticulate](https://rstudio.github.io/reticulate/) is an R library that lets you do the opposite: call Python directly from inside R

## If you're interested...

- Data visualization with Altair:

    J. Heer, D. Moritz, J. VanderPlas, B. Craft. *University of Washington Data Visualization Curriculum.* Set of Jupyter notebooks. [[link]](https://github.com/uwdata/visualization-curriculum)<br><br>

- Data visualization and wrangling concepts, taught through R:

    H. Wickham, G. Grolemund. *R for Data Science.* Electronic book, physical copy published by O'Reilly, 2017. [[link]](https://r4ds.had.co.nz/)<br><br>

- Altair documentation [[link]](https://altair-viz.github.io/)<br><br>

- Pandas documentation [[link]](https://pandas.pydata.org/docs/)

- Course website with materials:

    https://www.usna.edu/Users/math/uhan/sa463a/